In [1]:
from huggingface_hub import snapshot_download
from glob import glob
import pandas as pd

In [2]:
snapshot_download(repo_id="mesolitica/Malaysian-Reasoning", allow_patterns="data/*",
                 repo_type ='dataset', local_dir = './reasoning')

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

'/home/husein/ssd4/llama2/reasoning'

In [3]:
# !wget https://huggingface.co/datasets/mesolitica/Malaysian-SFT/resolve/main/extra/qwq-magpie.json
# !wget https://huggingface.co/datasets/mesolitica/Malaysian-SFT/resolve/main/extra/filter-qwq-500k-math.json

In [4]:
reasoning_en = 'You are going to enter reasoning mode, first you try to think step-by-step after that give the final answer.'
reasoning_ms = 'You are going to enter reasoning mode, first you try to think step-by-step in malay after that give the final answer.'

In [5]:
import json

with open('qwq-magpie.json') as fopen:
    d = json.load(fopen)
    
len(d)

7348

In [6]:
instructions = []
for d_ in d:
    instructions.append([
        {'role': 'system', 'content': reasoning_en},
        {'role': 'user', 'content': d_['problem']},
        {'role': 'assistant', 'content': d_['outputs']}
    ])

In [7]:
with open('filter-qwq-500k-math.json') as fopen:
    d = json.load(fopen)
    
len(d)

34805

In [8]:
d[0]

{'prompt': 'Derive the joint density function of the DE and SCF biases (f(DE, SCF)) using a Bayesian approach, assuming that the parameters α, β, λ, and θ follow normal distributions with known means and variances, and that the error terms ε and η follow normal distributions with zero means and unknown variances. Use the derived joint density function to compute the expected value of the portfolio return (E(S_t)) and the expected value of the portfolio volatility (E(σ)) across all investors.',
 'response': "So I've got this research problem here in behavioral finance, and it's all about how cognitive biases affect investment decisions. The researcher is looking at two main biases: the disposition effect (DE) and the sunk cost fallacy (SCF). She's got data on 1000 investors and their portfolios, which are modeled using a geometric Brownian motion (GBM).\n\nFirst, I need to understand what these biases are. The disposition effect is when investors sell their winning stocks too early and 

In [9]:
for d_ in d:
    instructions.append([
        {'role': 'system', 'content': reasoning_en},
        {'role': 'user', 'content': d_['prompt']},
        {'role': 'assistant', 'content': d_['response']}
    ])

In [10]:
glob('reasoning/data/*.parquet')

['reasoning/data/leetcode_hard-00000-of-00001.parquet',
 'reasoning/data/mallm-00000-of-00001.parquet',
 'reasoning/data/tatabahasa-00000-of-00001.parquet',
 'reasoning/data/maktabalbahri-00000-of-00001.parquet']

In [11]:
df = pd.read_parquet('reasoning/data/leetcode_hard-00000-of-00001.parquet')
df.head()

,id,slug,title,difficulty,content,solution,language,english,malay
0,1568,minimum-number-of-days-to-disconnect-island,Minimum Number of Days to Disconnect Island,Hard,You are given an `m x n` binary grid `grid` wh...,"```python\ndef pseudoPalindromicPaths(root, cn...",python,"Alright, so I've got this problem here about a...","Baiklah, jadi saya mempunyai masalah ini di si..."
1,1714,sum-of-special-evenly-spaced-elements-in-array,Sum Of Special Evenly-Spaced Elements In Array,Hard,You are given a **0-indexed** integer array `n...,```javascript\nfunction reorderSpaces(text) {\...,javascript,"Alright, I've got this problem here. So, I hav...","Baiklah, saya ada masalah ini di sini. Jadi, s..."
2,1326,minimum-number-of-taps-to-open-to-water-a-garden,Minimum Number of Taps to Open to Water a Garden,Hard,There is a one-dimensional garden on the x-axi...,```javascript\nfunction sumOfFlooredPairs(nums...,javascript,"Alright, I've got this problem about watering ...","Baiklah, saya mempunyai masalah ini tentang me..."
3,656,coin-path,Coin Path,Hard,You are given an integer array `coins` (**1-in...,```cpp\n#include <vector>\n#include <queue>\n#...,c++,"Alright, I've got this problem here. Let's see...","Baiklah, saya mempunyai masalah ini di sini. M..."
4,410,split-array-largest-sum,Split Array Largest Sum,Hard,Given an integer array `nums` and an integer `...,"```python\ndef splitArray(nums, m):\n left,...",python,"Alright, so I've got this problem here. I need...","Baiklah, jadi saya mempunyai masalah ini di si..."


In [12]:
for i in range(len(df)):
    instructions.append([
        {'role': 'system', 'content': reasoning_en},
        {'role': 'user', 'content': df.iloc[i]['content'] + f"\n\nsolve the problem using {df.iloc[i]['language']}"},
        {'role': 'assistant', 'content': df.iloc[i]['english']}
    ])
    
    instructions.append([
        {'role': 'system', 'content': reasoning_ms},
        {'role': 'user', 'content': df.iloc[i]['content'] + f"\n\nsolve the problem using {df.iloc[i]['language']}"},
        {'role': 'assistant', 'content': df.iloc[i]['malay']}
    ])
    
len(instructions)

45275

In [13]:
df = pd.read_parquet('reasoning/data/mallm-00000-of-00001.parquet')
df.head()

,content,solution,malay
0,subject: chemistry\nPernyataan yang manakah me...,B,Saya ada soalan tentang asid kuat dalam kimia....
1,subject: chemistry\nApakah yang dimaksudkan de...,A,Saya perlu menjawab soalan ini tentang elektro...
2,subject: islamic_study\nHikmah perkahwinan ial...,B,Saya sedang cuba menjawab soalan ini tentang h...
3,subject: business_study\nEncik Azri dan Encik ...,C,Encik Azri dan Wahid ingin menubuhkan syarikat...
4,subject: malay_language\nPilih penjodoh bilang...,C,Saya perlu memilih penjodoh bilangan yang pali...


In [14]:
for i in range(len(df)):
    if 'subject: islamic_study' in df.iloc[i]['content']:
        continue
    q = df.iloc[i]['content'].split('\n')
    q = '\n'.join(q[1:])
    
    instructions.append([
        {'role': 'system', 'content': reasoning_ms},
        {'role': 'user', 'content': q},
        {'role': 'assistant', 'content': df.iloc[i]['malay']}
    ])
    
len(instructions)

50818

In [15]:
df = pd.read_parquet('reasoning/data/tatabahasa-00000-of-00001.parquet')
df.head()

,content,solution,malay
0,Isikan tempat kosong dengan kata bantu yang se...,A,Saya perlu mengisi tempat kosong dalam ayat in...
1,Pilih penggunaan kata penguat yang betul.\nPud...,B,Saya perlu memilih kata penguat yang paling se...
2,Isikan tempat kosong dengan kata bantu yang se...,A,Saya ada soalan tentang pilihan perkataan yang...
3,Isi tempat kosong dengan jawapan yang paling s...,D,Saya perlu memilih jawapan yang paling sesuai ...
4,Isi tempat kosong dalam ayat-ayat di bawah den...,A,Saya perlu mengisi tempat kosong dalam ayat in...


In [16]:
for i in range(len(df)):
    
    instructions.append([
        {'role': 'system', 'content': reasoning_ms},
        {'role': 'user', 'content': df.iloc[i]['content']},
        {'role': 'assistant', 'content': df.iloc[i]['malay']}
    ])
    
len(instructions)

52081

In [17]:
df = pd.read_parquet('reasoning/data/maktabalbahri-00000-of-00001.parquet')
df.head()

,content,malay
0,Adakah benar syair “Tola’al Baru ‘Alaina” disy...,"Alhamdulillah, puji dan syukur kepada Ilahi at..."
1,"Asslamualaikum Ustaz, Recently, my employer in...","If you know for certain that it is money, for ..."
2,Assalamualaikum Dato’ Seri. Apa yang perlu say...,"Waalaikumusslam wbt.\n\nAlhamdulillah, segala..."
3,Apakah hukum menggunakan duit biasasiswa pelaj...,"Alhamdulillah, puji dan syukur kepada Ilahi at..."
4,What is the ruling of using one’s scholarship ...,"Alhamdulillah, praise and gratitude to Allah S..."


In [18]:
from huggingface_hub import hf_hub_download
import fasttext

filename = hf_hub_download(
    repo_id="mesolitica/fasttext-language-detection-bahasa-en", 
    filename="fasttext.ftz"
)
lang_model = fasttext.load_model(filename)

In [19]:
for i in range(len(df)):
    if lang_model.predict(df.iloc[i]['content'].replace('\n', ' '))[0][0] == '__label__english':
        system = reasoning_en
    else:
        system = reasoning_ms
    
    instructions.append([
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': df.iloc[i]['content'].replace('\xa0', ' ')},
        {'role': 'assistant', 'content': df.iloc[i]['malay'].replace('\xa0', ' ')}
    ])
    
len(instructions)

59560

In [20]:
instructions[0]

[{'role': 'system',
  'content': 'You are going to enter reasoning mode, first you try to think step-by-step after that give the final answer.'},
 {'role': 'user',
  'content': 'Design a highly available and scalable web application architecture on AWS that incorporates auto-scaling, load balancing, and data replication across multiple regions to ensure low latency and high durability. Additionally, implement a DDoS protection mechanism and a backup strategy that allows for rapid recovery in case of a service disruption.'},
 {'role': 'assistant',
  'content': "Designing a highly available and scalable web application architecture on AWS involves several key components and considerations. I'll walk through the process step by step, incorporating auto-scaling, load balancing, data replication, DDoS protection, and a backup strategy.\n\n### 1. **Choosing the Right Services**\n\nFirst, I need to select the appropriate AWS services that will help me achieve high availability, scalability, a

In [21]:
with open('combined-malaysian-reasoning.jsonl', 'w') as fopen:
    for i in instructions:
        fopen.write(f'{json.dumps(i)}\n')

In [22]:
!ls -lh combined-malaysian-reasoning.jsonl

-rw-rw-r-- 1 husein husein 1.1G Jan   6 13:01 combined-malaysian-reasoning.jsonl


In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="combined-malaysian-reasoning.jsonl",
    path_in_repo="combine/combined-malaysian-reasoning.jsonl",
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

combined-malaysian-reasoning.jsonl:   0%|          | 0.00/1.15G [00:00<?, ?B/s]